In [1]:
import pandas as pd
import numpy as np 
import datetime
import os

def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)
            
start_date_2019Q3=datetime.date(2019,8,4)
end_date_2019Q3=datetime.date(2019,11,2)

In [4]:
# POS date

list_all_files=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
list_all_files=[x for x in list_all_files if "daily" in x.lower() and x[-4:]==".txt"]
list_all_files=[x for x in list_all_files if x.split("/MediaStorm_")[1][:10]>=str(start_date_2019Q3)]
list_all_files=[x for x in list_all_files if x.split("/MediaStorm_")[1][:10]<=str(end_date_2019Q3)]
print(len(list_all_files))
list_all_files.sort()

13


In [5]:
df_all_POS_by_store=pd.DataFrame()
for file in list_all_files:
    df=pd.read_csv(file,dtype=str,sep="|")
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df_rewards=df[pd.notnull(df['customer_id_hashed'])]
    df_non_rewards=df[pd.isnull(df['customer_id_hashed'])]
    
    df_rewards_sales=df_rewards.groupby(['location_id','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"rewards_sales"})
    df_rewards_trans=df_rewards[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_rewards_trans['rewards_trans']=1
    df_rewards_trans=df_rewards_trans.groupby(['location_id','transaction_dt'])['rewards_trans'].sum().to_frame().reset_index()
    df_rewards=pd.merge(df_rewards_sales,df_rewards_trans,on=['location_id','transaction_dt'],how="outer")
    
    df_non_rewards_sales=df_non_rewards.groupby(['location_id','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"non_rewards_sales"})
    df_non_rewards_trans=df_non_rewards[['location_id','transaction_dt','transaction_id']].drop_duplicates()
    df_non_rewards_trans['non_rewards_trans']=1
    df_non_rewards_trans=df_non_rewards_trans.groupby(['location_id','transaction_dt'])['non_rewards_trans'].sum().to_frame().reset_index()
    df_non_rewards=pd.merge(df_non_rewards_sales,df_non_rewards_trans,on=['location_id','transaction_dt'],how="outer")
    
    df=pd.merge(df_rewards,df_non_rewards,on=['location_id','transaction_dt'],how="outer")
    df['week_end_dt']=df['transaction_dt'].max()
    df_all_POS_by_store=df_all_POS_by_store.append(df)
    print(datetime.datetime.now(),df['transaction_dt'].max())


2019-12-23 14:54:55.641605 2019-08-10
2019-12-23 14:58:05.243542 2019-08-17
2019-12-23 15:01:16.258377 2019-08-24
2019-12-23 15:04:48.076104 2019-08-31
2019-12-23 15:06:20.328205 2019-09-07
2019-12-23 15:09:57.923454 2019-09-14
2019-12-23 15:13:01.632319 2019-09-21
2019-12-23 15:17:50.553908 2019-09-28
2019-12-23 15:21:25.103801 2019-10-05
2019-12-23 15:22:45.381834 2019-10-12
2019-12-23 15:28:00.083671 2019-10-19
2019-12-23 15:29:25.697554 2019-10-26
2019-12-23 15:33:04.131337 2019-11-02


In [6]:
df_all_POS_by_store['store_type']=np.where(df_all_POS_by_store['location_id']=="6990","online","instore")
df_summary=df_all_POS_by_store.groupby(["week_end_dt",'store_type'])['rewards_sales','non_rewards_sales','rewards_trans','non_rewards_trans'].sum().reset_index()

In [7]:
writer=pd.ExcelWriter("./BL_total_sales_2019Q3_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_summary.to_excel(writer,"df_summary",index=False)
df_all_POS_by_store.to_excel(writer,"df_by_store_date",index=False)
writer.save()

In [8]:
df_all_POS_by_store.shape

(128736, 8)

In [9]:
df_all_POS_by_store.head(3)

,location_id,transaction_dt,rewards_sales,rewards_trans,non_rewards_sales,non_rewards_trans,week_end_dt,store_type
0,1,2019-08-04,6136.13,157.0,2930.72,150,2019-08-10,instore
1,1,2019-08-05,4661.49,135.0,3089.69,134,2019-08-10,instore
2,1,2019-08-06,4065.35,131.0,2628.10,121,2019-08-10,instore
